# Subset the data to only include potential rows that could have discrepancies

Our method to subset the data so we don't need to look through all the rows to find the potential false positives is two-fold:

1. We filter by rows contains a set of keywords from the honors thesis.

2. We filter by rows that have sentiment score of 4 or lower (5 being the most positive sentiment score).

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

c:\Users\grego\Documents\GitHub\DS440CapstoneIndubitably\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Define filters

In [9]:
def keyword_filter(row: pd.Series) -> bool:
    """Return True if the row contains a keyword in the list of keywords"""
    keywords = [
        'deceiving', 
        "decieving", # I bet there's typos
        'disappointing', 
        'horrible',
        'terrible',
        'awful',
        'bad',
        "misleading",
        "inaccurate",
        "incorrect"]
    if not isinstance(row['comments'], str):
        return False
    return any([keyword in row['comments'] for keyword in keywords])

def sentiment_filter(row: pd.Series) -> bool:
    pass
    # for i in range(1,len(comments)):
    # comment = comments[i]
    # tokens = tokenizer.encode(comment, return_tensors='pt')
    # result = model(tokens)
    # pred = int(torch.argmax(result.logits))+1

def apply_filters(raw_fp: str, filtered_fp: str) -> None:
    """Apply the filters to the raw data and save the result to a new file"""

    print("Reading data from", raw_fp)
    data = pd.read_csv(raw_fp)

    original_len = len(data)

    print("Number of rows:", original_len)

    print("Applying keyword filter...")
    data = data[data.apply(keyword_filter, axis=1)]
    print("Number of remaining rows:", len(data))

    # print("Applying sentiment filter...")
    # data = data[data.apply(sentiment_filter, axis=1)]
    # print("Number of remaining rows:", len(data))

    print("Data to look through reduced by", (1 - (len(data) / original_len)) * 100, "%")

    print("Saving filtered data to", filtered_fp)
    data.to_csv(filtered_fp, index=False)

## Apply filters and save results to /data/filtered

In [10]:
apply_filters("../data/raw/texas_reviews.csv", "../data/filtered/texas_reviews.csv")

Reading data from ../data/raw/texas_reviews.csv
Number of rows: 332098
Applying keyword filter...
Number of remaining rows: 2546
Data to look through reduced by 99.23335882781589 %
Saving filtered data to ../data/filtered/texas_reviews.csv


In [12]:
apply_filters("../data/raw/florida_reviews.csv", "../data/filtered/florida_reviews.csv")

Reading data from ../data/raw/florida_reviews.csv
Number of rows: 195857
Applying keyword filter...
Number of remaining rows: 1996
Data to look through reduced by 98.98088911808105 %
Saving filtered data to ../data/filtered/florida_reviews.csv


In [13]:
apply_filters("../data/raw/california_reviews.csv", "../data/filtered/california_reviews.csv")

Reading data from ../data/raw/california_reviews.csv
Number of rows: 366643
Applying keyword filter...
Number of remaining rows: 3174
Data to look through reduced by 99.1343077598645 %
Saving filtered data to ../data/filtered/california_reviews.csv
